In [ ]:
from IPython.display import clear_output

from deap import base
from deap import creator
from deap import tools

from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [ ]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=15, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# All of these variables substantially change calculation time
stk_object.dt = 30
n_targets = 15
n_sats = 12
n_pop = 10
n_gen = 0

In [ ]:
# Generating Targets

# Generating a polygon to bound the lat/lon coordinates, you can create your polygon, in the same format as Targets_Polygon.txt.
poly = Create_Poly('Input_Files/Targets_Polygon.txt')

# Writing random points within the polygon to a target file.
targets_filename = 'Input_Files/Targets_File.txt'
polygon_random_points(poly,n_targets).to_csv(targets_filename,index=False)

# Loading targets into stk from file.
targets_filename = 'Input_Files/Targets_File.txt'
stk_object.Target_Loader(targets_filename)

# Plotting the polygon and generated targets on the map.
plot_targets_and_polygon(poly,targets_filename)

In [ ]:
# Defining cost function

def satellite_cost(Individual):
    Alt = Individual[0]
    Inc = Individual[1]
    num_sats = int(Individual[2])
    num_planes = int(Individual[3])
    if num_planes <= num_sats:
        file = open("Input_Files/Satellites_File.txt","w")
        file.write("Per,Apo,Inc,AoP,Asc,Loc,Tar\n")
        sats = num_sats*[1]
        planes = np.array_split(sats,num_planes)
        Asc = 0
        for plane in planes:
            Loc = 0
            for sat in plane:
                file.write(f"{Alt},{Alt},{Inc},{0},{round(Asc,4)},{round(Loc,4)},{1}\n")
                if len(plane)>1: Loc += 360/(len(plane)-1)
            if len(planes)>1:Asc += 180/(len(planes)-1)
        file.close()
        satellites_filename = 'Input_Files/Satellites_File.txt'
        stk_object.Satellite_Loader(satellites_filename)
        stk_object.Compute_AzEl()
        stk_object.Sort_AzEl()
        percentages = []
        times = []
        df = stk_object.AzEl_4d_representation
        for idx,tar in enumerate(stk_object.targets):
            window = df['Target'].values==idx+1
            target_percentages = df['Target Percentage'].values[window]
            target_times = df['Time'].values[window]
            if len(target_percentages)>0:
                end_idx = target_percentages==target_percentages[-1]
                percentages.append(target_percentages[-1])
                times.append(target_times[end_idx][0])
            else:
                percentages.append(0)
                times.append(stk_object.root.CurrentScenario.StopTime)
        return sum(percentages)/len(percentages),max(times)/3600,num_sats
    else:
        return -100,stk_object.root.CurrentScenario.StopTime,100

In [ ]:
# Running Optimization

# Creating DEAP optimization model (positive weights to maximize, negative weights to minimize)
creator.create("FitnessMax", base.Fitness, weights=(7.0,-1.0,-2.0))
# Creating satellite for the model
creator.create("Satellite", list, fitness=creator.FitnessMax)

# Lower and Upper Bounds of Variables
lower = [580,0,1,1]
upper = [800,180,n_sats,n_sats]

# Registering variables to the satellite
toolbox = base.Toolbox()
toolbox.register("attr_alt", random.randint, lower[0], upper[0])
toolbox.register("attr_inc", random.randint, lower[1], upper[1])
toolbox.register("attr_num_planes", random.randint, lower[2], upper[2])
toolbox.register("attr_sat_per_plane", random.randint, lower[3], upper[3])

# Registering satellite to the model
toolbox.register("satellite", tools.initCycle, creator.Satellite,
                 (toolbox.attr_alt,
                  toolbox.attr_inc,
                  toolbox.attr_num_planes,
                  toolbox.attr_sat_per_plane), n=1)

# Registering tools for the algorithm
toolbox.register("population", tools.initRepeat, list, toolbox.satellite)
toolbox.register("evaluate", satellite_cost)
# toolbox.register("mate", tools.cxBlend, alpha=0.1)
toolbox.register("mate", tools.cxSimulatedBinaryBounded,eta=0.25,low=n_sats*lower,up=n_sats*upper)
# toolbox.register("mutate", gp.mutEphemeral,mode="one")
toolbox.register("mutate", tools.mutPolynomialBounded,eta=0.25,low=n_sats*lower,up=n_sats*upper,indpb=0.5)
toolbox.register("select", tools.selTournament, tournsize=3)

clear_output(wait=True)

g = 0

# Creating a population to evolve
pop = toolbox.population(n=n_pop)
print("-- Generation %i --" % g)
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit
CXPB, MUTPB = 0.7, 0.3
fits = [ind.fitness.values for ind in pop]
hof = tools.HallOfFame(5)
hof.update(pop)

# Begin the evolution
while g < n_gen:
    clear_output(wait=True)
    if len(hof)>0:
        print("HOF so far...")
        print(f"{hof[0].fitness.values[0]} percent imaged")
        print(f"{hof[0].fitness.values[1]} hours to image")
        print(f"{hof[0].fitness.values[2]} satellites")
    g = g + 1
    print("-- Generation %i --" % g)
    # A new generation
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)

    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    pop[:] = offspring
    hof.update(pop)
print(hof[0])
print(satellite_cost(hof[0]))

In [ ]:
# Computing best data at higher dt

stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=15, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")
stk_object.dt = 2.5

percent,time,sats = satellite_cost(hof[0])
print("\nBest run with higher dt")
print(f"Bins filled     : {percent:.2f}%")
print(f"Hours to image  : {time:.2f}")
print(f"Satellites used : {sats:.2f}")
df = stk_object.AzEl_4d_representation
for idx,tar in enumerate(stk_object.targets):
    window = df['Target'].values==idx+1
    target_percentages = df['Target Percentage'].values[window]
    target_times = df['Time'].values[window]
    if len(target_percentages)>0:
        end_idx = target_percentages==target_percentages[-1]
        print(f"{tar}->{target_percentages[end_idx][0]:.2f}%,{target_times[end_idx][0]/3600:.2f} hours")

In [ ]:
# Plotting optimization data

planning_data = stk_object.AzEl_4d_representation

fig = px.scatter_3d(planning_data,x='Time',y='Target',z='Bin',color='Point')

fig.update_traces(marker=dict(size=2))
fig.update_layout(title='4d representation of optimization')

fig.show()

In [ ]:
# Plotting target-point data

target_data = np.array([stk_object.Targets_Point_Bins[tar].T for tar in stk_object.targets])

fig = px.imshow(target_data,
                animation_frame=0,
                y=[f"{10*idx}-{10*(idx+1)}" for idx in range(9)],
                x=[f"{10*idx}-{10*(idx+1)}" for idx in range(36)])

fig.show()

In [ ]:
# import scipy.interpolate as interpolate

# interpolated_df = {'Time':[],'Satellite':[],'Target':[],'Azimuth':[],'Elevation':[],'Group':[]}
# for sat_num,sat in enumerate(stk_object.satellites):
#     sat_window = stk_object.AzEl_data['Satellite'] == sat_num+1
#     for tar_num,tar in enumerate(stk_object.targets):
#         tar_window = stk_object.AzEl_data['Target'] == tar_num+1
#         for group in np.unique(stk_object.AzEl_data['Group'].values):
#             group_window = stk_object.AzEl_data['Group'].values==group
#             group_df = stk_object.AzEl_data[sat_window&tar_window&group_window]
#             time = group_df['Time'].values
#             az = group_df['Azimuth'].values
#             el = group_df['Elevation'].values
#             if len(np.unique(time))==len(time) and len(time) > 0:
#                 # az_t = interpolate.CubicSpline(x=time,y=az)
#                 # el_t = interpolate.CubicSpline(x=time,y=el)
#                 az_t = interpolate.interp1d(x=time,y=az)
#                 el_t = interpolate.interp1d(x=time,y=el)
#                 times = np.arange(time[0]+0.01,time[-1]-0.01,5)
#                 for t in times:
#                     interpolated_df['Time'].append(t)
#                     interpolated_df['Satellite'].append(sat_num+1)
#                     interpolated_df['Target'].append(tar_num+1)
#                     interpolated_df['Azimuth'].append(az_t(t))
#                     interpolated_df['Elevation'].append(el_t(t))
#                     interpolated_df['Group'].append(group)
# stk_object.AzEl_data = pd.concat([stk_object.AzEl_data,pd.DataFrame(interpolated_df)],ignore_index=True).sort_values(by='Time')            

In [ ]:
# Plotting optimization data

AzEl_data = stk_object.AzEl_data

fig = px.scatter(AzEl_data,x='Azimuth',y='Elevation',color='Satellite',animation_frame='Target',hover_data='Time')

fig.update_traces(marker=dict(size=2))
fig.update_layout(title='Elevation vs Azimuth')

fig.show()

In [ ]:
stk_object.Sort_AzEl()
percentages = []
times = []
df = stk_object.AzEl_4d_representation
for idx,tar in enumerate(stk_object.targets):
    window = df['Target'].values==idx+1
    target_percentages = df['Target Percentage'].values[window]
    if len(target_percentages)>0:
        end_idx = df['Target Percentage'].values==target_percentages[-1]
        percentages.append(df['Target Percentage'].values[end_idx][0])
        times.append(df['Time'].values[end_idx][0])
    else:
        percentages.append(0)
        times.append(stk_object.root.CurrentScenario.StopTime)

percent,time = sum(percentages)/len(percentages),max(times)/3600

print("\nBest run with interpolation")
print(f"Bins filled    : {percent:.2f}%")
print(f"Hours to image : {time:.2f}")